In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import requests
from tqdm import tqdm_notebook
import glob

In [3]:
os.listdir('../')

['.DS_Store',
 'requirements.txt',
 'synonyms',
 'yappy_search',
 '.gitignore',
 '.git',
 'notebooks']

In [4]:
# Считываем данные из таблицы (CSV или Excel, в зависимости от формата)
file_path = '../../yappy_hackaton_2024_400k.csv'  # укажите правильный путь к файлу
df = pd.read_csv(file_path)  # если у вас Excel, используйте pd.read_excel(file_path)

In [5]:
df

,link,description
0,https://cdn-st.rutubelist.ru/media/b0/e9/ef285...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m..."
1,https://cdn-st.rutubelist.ru/media/39/6c/b31bc...,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #rob...
2,https://cdn-st.rutubelist.ru/media/e9/e0/b47a9...,"#boobs , #красивыедевушки , #ass"
3,https://cdn-st.rutubelist.ru/media/87/43/b11df...,NaN
4,https://cdn-st.rutubelist.ru/media/d1/e7/642dc...,NaN
...,...,...
399995,https://cdn-st.rutubelist.ru/media/dd/6a/ac296...,#ceramics #handmade #ceramic #clay #ceramicscu...
399996,https://cdn-st.rutubelist.ru/media/aa/d3/4b4c3...,#красивыедевушки #красотки #грудь #boobs #секс...
399997,https://cdn-st.rutubelist.ru/media/73/05/3f80f...,"#лайфхаки , #эксперименты , #roblox , #игрушки..."
399998,https://cdn-st.rutubelist.ru/media/f8/2b/320f7...,#образ #lookbook #показ #неделямоды #капсула #...


In [6]:
import sys

sys.path.insert(
    0,
    "/Users/andrey/Documents/hacks/Yappy/git/",
)

In [7]:
from yappy_search.utils import  download_video

In [8]:
dir_w_video:str = '../../data_video/'

In [9]:
os.makedirs(dir_w_video,exist_ok=True)

In [10]:
# Скачиваем все видео из списка
for index, row in tqdm_notebook(df.iterrows()):
    video_url = row['link']
    description = row['description']
    video_name = f"{index}{'_'.join(video_url.split('media')[1].split('/'))}"  # можно использовать описание для имени файла
    video_path = os.path.join(dir_w_video,video_name)
    if os.path.isfile(video_path):
        continue
    response = download_video(video_url, video_path)
    print(f"Downloaded {video_name}.")
    if index > 150:
        break
print("All videos downloaded successfully.")

/var/folders/yr/mmqsh_3x4z92c40wn0j7x2yw0000gp/T/ipykernel_11095/2508132707.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm_notebook(df.iterrows()):


0it [00:00, ?it/s]

Downloaded 157_65_7b_2cb8f34c49afa12e45b6d2f44e28_fhd.mp4.
All videos downloaded successfully.


In [11]:
video_name

'157_65_7b_2cb8f34c49afa12e45b6d2f44e28_fhd.mp4'

In [12]:
response.headers

{'Server': 'Angie', 'Date': 'Fri, 07 Jun 2024 18:26:56 GMT', 'Content-Type': 'video/mp4', 'Content-Length': '3131177', 'Connection': 'keep-alive', 'Accept-Ranges': 'bytes', 'Last-Modified': 'Fri, 15 Sep 2023 15:34:13 GMT', 'x-rgw-object-type': 'Normal', 'ETag': '"9498cfd3ce9ae70da1885857dfbafd78"', 'x-amz-request-id': 'tx00000ffd648f29b4947cd-00666350f0-3a409c50-ritm', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload'}

In [13]:
response.history

[]

In [14]:
response.__sizeof__()

24

In [15]:
# Получение списка всех файлов с расширением .mp4 в директории
video_files = glob.glob(os.path.join(dir_w_video, '*.mp4'))
# Сортировка списка файлов по первому значению при split('_')
sorted_video_files = sorted(video_files, key=lambda x: int(os.path.basename(x).split('_')[0]))

# Вывод отсортированного списка файлов
for video_file in sorted_video_files[:5]:
    print(video_file)

../../data_video/0_b0_e9_ef285e0241139fc611318ed33071_fhd.mp4
../../data_video/1_39_6c_b31bc6864bef9d8a96814f1822ca_fhd.mp4
../../data_video/2_e9_e0_b47a9df14a5e97942715e5e705c0_fhd.mp4
../../data_video/3_87_43_b11df3f344d0af773aac81e410ee_fhd.mp4
../../data_video/4_d1_e7_642dc2194fcdb69664f832d5f2dd_fhd.mp4


In [16]:
dir_w_screens = '../../scrins/'
# Получение списка всех файлов с расширением .mp4 в директории
screens_files = glob.glob(os.path.join(dir_w_screens, '*.png'))
# Сортировка списка файлов по первому значению при split('_')
# sorted_screens_files = sorted(screens_files, key=lambda x: int(os.path.basename(x).split('_')[0]))

# Вывод отсортированного списка файлов
for screen_file in screens_files[:5]:
    print(screen_file)

../../scrins/Снимок экрана 2024-06-03 в 20.59.20.png
../../scrins/Снимок экрана 2024-06-03 в 20.59.35.png
../../scrins/Снимок экрана 2024-06-03 в 20.59.46.png
../../scrins/Снимок экрана 2024-06-03 в 20.59.53.png


## проверяем pipeline работы

In [17]:
import torch
import asyncio

In [18]:
from yappy_search.config import ConfigImageCaptioning
from yappy_search.model import VideoProcessor

In [19]:
config = ConfigImageCaptioning()

In [20]:
config

ConfigImageCaptioning(min_length=0, max_length=100, num_beams=4, model_name_image_caption='GeorgeBredis/ruIdefics2-ruLLaVA-merged', model_name_audio_whisper='small', model_name_audio_lang='ru')

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
device

device(type='cpu')

In [23]:
model_all = VideoProcessor(config,device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [24]:
video_path = sorted_video_files[0]

In [25]:
audio_output_dir:str|os.PathLike = os.path.join(video_path.rsplit('/',2)[0],'data_audio')
os.makedirs(audio_output_dir, exist_ok=True)

In [27]:
transcriptions = []
recognitions = []
for video_path in tqdm_notebook(sorted_video_files[:2]):
    audio_path = model_all.audio_transcription.extract_audio(video_path, audio_output_dir)
    transcription = model_all.audio_transcription.transcribe(audio_path)
    
    loop = asyncio.get_event_loop()
    recognition = loop.run_until_complete(model_all.song_recognition.recognize_audio(audio_path))

    transcriptions.append(transcription)
    recognitions.append(recognition)

  0%|          | 0/2 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [28]:
transcription

" Сейчас я тебе покажу секретную команду в Roblox'е! Чтобы её активировать поставь лайк и подпишись, а также в виде в чат команду I love you! Когда вы её ведёте у вас на экране появится вот такой клоун! Скривц был создан одним из создателей Roblox'а и работает только в эракде есть Roblox Developer Service. Чтобы в Раскриме ранжа всего-то выйти из игры, но лучше не проверяй!"

In [29]:
recognition

'Song not recognized'

In [ ]:
%%time
captions = model_all.image_captioning.generate_caption(video_path)

In [ ]:
captions

In [ ]:
1+1

In [ ]:
for index in range(len(transcriptions)):
    row = df.iloc[index]
    print(index)
    print(f'link:{row.link}')
    print(f'description:{row.description}')
    print(f'transcription:{transcriptions[index]}')
    print(f'recognition:{recognitions[index]}')
    print(20*'-')

In [ ]:
# !pip install -U transformers

In [ ]:
# !pip install av bitsandbytes moviepy openai-whisper

In [ ]:
# !pip install accelerate

In [ ]:
import numpy as np
import av
# from peft import LoraConfig
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics2ForConditionalGeneration,AutoModelForVision2Seq
from transformers.image_utils import load_image
import torch
from PIL import Image

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

#### Проба [video LLaVa](https://huggingface.co/spaces/LanguageBind/Video-LLaVA)
[git](https://github.com/PKU-YuanGroup/Video-LLaVA)

In [ ]:
# !pip install -U transformers

In [ ]:
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration

In [ ]:
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
model = VideoLlavaForConditionalGeneration.from_pretrained(
    "LanguageBind/Video-LLaVA-7B-hf",
     torch_dtype=torch.float16,
     quantization_config=quantization_config,
)
processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")

RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
video_captions = []
for video_path in tqdm_notebook(sorted_video_files[:20]):
    prompt = "USER: <video>Make a summary of video. ASSISTANT:"
    container = av.open(video_path)

    # sample uniformly 8 frames from the video
    total_frames = container.streams.video[0].frames
    indices = np.arange(0, total_frames, total_frames / 8).astype(int)
    # indices = [int(0.25 * total_frames), int(0.5 * total_frames), int(0.75 * total_frames)]
    clip = read_video_pyav(container, indices)
    inputs = processor(text=prompt, videos=clip, return_tensors="pt")

    # Generate
    generate_ids = model.generate(**inputs, max_length=80)
    caption = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(caption)
    break

/var/folders/yr/mmqsh_3x4z92c40wn0j7x2yw0000gp/T/ipykernel_39766/235269379.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for video_path in tqdm_notebook(sorted_video_files[:20]):


  0%|          | 0/20 [00:00<?, ?it/s]

NameError: name 'processor' is not defined

In [ ]:
model_name = "GuardisAI/Video-LLaVA-7B-GPTQ-4bit-V1"
#You are using a model of type llava to instantiate a model of type video_llava. This is not supported for all configurations of models and can yield errors.

### [ruIdefics2-ruLLaVA-merged](https://huggingface.co/GeorgeBredis/ruIdefics2-ruLLaVA-merged)

In [ ]:
DEVICE  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
processor = AutoProcessor.from_pretrained(
    "GeorgeBredis/ruIdefics2-ruLLaVA-merged",
    do_image_splitting=False
)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

model = Idefics2ForConditionalGeneration.from_pretrained(
     "GeorgeBredis/ruIdefics2-ruLLaVA-merged",
     torch_dtype=torch.float16,
     quantization_config=quantization_config,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Что изображено на данной картинке?"},
        ]
    }
]

In [ ]:
video_captions = []
for video_path in tqdm_notebook(sorted_video_files):
    # load video
    container = av.open(video_path)

    # get fps (corrected approach)
    fps = container.streams.video[0].average_rate.numerator / container.streams.video[0].average_rate.denominator

    # extract frames at 25%, 50%, & 75%
    seg_len = container.streams.video[0].frames  # total number of frames
    indices = [int(0.25 * seg_len), int(0.5 * seg_len), int(0.75 * seg_len)]
    frames = []

    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i in indices:
          frames.append(frame.to_ndarray(format="rgb24"))
    # generate caption
    gen_kwargs = {
        "min_length": 0,
        "max_length": 100,
        "num_beams": 4,
    }

    # Convert numpy arrays to PIL Images
    pil_images = [Image.fromarray(frame) for frame in frames]

    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=pil_images[:1], return_tensors="pt")
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    generated_ids = model.generate(**inputs, max_new_tokens=100)
    generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
    # caption = '|'.join([pred.strip() for pred in preds])
    caption = generated_texts
    video_captions.append(caption)

<ipython-input-27-5f37e01cd0ee>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for video_path in tqdm_notebook(sorted_video_files):


  0%|          | 0/27 [00:00<?, ?it/s]

InvalidDataError: [Errno 1094995529] Invalid data found when processing input

In [ ]:
for index in range(len(video_captions)):
    row = df.iloc[index]
    print(f'index: {index}|link:{row.link}')
    print(f'description:{row.description}')
    print(f'video_caption:{video_captions[index][0]}')
    print(20*'-')

index: 0|link:https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4
description:#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin
video_caption:User: Что изображено на данной картинке? 
Assistant: На изображении представлен футбольный матч, где группа людей играет в футбол на поле. Они разбросаны по всему полю, некоторые из них стоят ближе к центру, а другие - ближе к краям.

Всего на поле шесть человек, которые активно участвуют в игре. Один из игроков находится в центре по
--------------------
index: 1|link:https://cdn-st.rutubelist.ru/media/39/6c/b31bc6864bef9d8a96814f1822ca/fhd.mp4
description:🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #roblox #роблокс
video_caption:User: Что изображено на данной картинке? 
Assistant: На изображении представлен игровой персонаж, стоящий на улице, который, похоже, является частью игры. Он одет в черно-белую форму и, похоже, является игровым персонажем.
--------------------
index: 2|link:h

### [RUDOLPH](https://habr.com/ru/companies/sberbank/articles/733470/)
RuCLIP старее

### [Chat-UniVi-7B-v1.5](https://huggingface.co/Chat-UniVi/Chat-UniVi-7B-v1.5/tree/main)

### [blip-2](https://huggingface.co/blog/blip-2)
[blip2-opt-2.7b](https://huggingface.co/Salesforce/blip2-opt-2.7b)

### [pix2struct](https://huggingface.co/google/pix2struct-textcaps-base)

### проба [SpaceTimeGPT - A Spatiotemporal Video Captioning Model](https://huggingface.co/Neleac/SpaceTimeGPT#spacetimegpt---a-spatiotemporal-video-captioning-model)

In [ ]:
import av
import numpy as np
import torch
from transformers import AutoImageProcessor, AutoTokenizer, VisionEncoderDecoderModel

device = "cuda" if torch.cuda.is_available() else "cpu"

# load pretrained processor, tokenizer, and model
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = VisionEncoderDecoderModel.from_pretrained("Neleac/timesformer-gpt2-video-captioning").to(device)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [ ]:
video_captions = []
for video_path in tqdm_notebook(sorted_video_files[:20]):
    # load video
    container = av.open(video_path)

    # extract evenly spaced frames from video
    ###
    # seg_len = container.streams.video[0].frames
    # clip_len = model.config.encoder.num_frames
    # indices = set(np.linspace(0, seg_len, num=clip_len, endpoint=False).astype(np.int64))

    frames = []
    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i in indices:
            frames.append(frame.to_ndarray(format="rgb24"))

    # generate caption
    gen_kwargs = {
        "min_length": 0,
        "max_length": 100,
        "num_beams": 4,
    }
    pixel_values = image_processor(frames, return_tensors="pt").pixel_values.to(device)
    tokens = model.generate(pixel_values, **gen_kwargs)
    caption = tokenizer.batch_decode(tokens, skip_special_tokens=True)[0]
    video_captions.append(caption)

  0%|          | 0/20 [00:00<?, ?it/s]

RuntimeError: shape '[1, 5, 14, 8, 768]' is invalid for input of size 451584

In [ ]:
indices

[140, 281, 421]

In [ ]:
caption

'A man and a woman are playing a video game together.'

In [ ]:
for index in range(len(video_captions)):
    row = df.iloc[index]
    print(f'link:{row.link}')
    print(f'description:{row.description}')
    print(f'video_caption:{video_captions[index]}')
    print(20*'-')

link:https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4
description:#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin
video_caption:A group of people are playing a game of soccer on a field.
--------------------
link:https://cdn-st.rutubelist.ru/media/39/6c/b31bc6864bef9d8a96814f1822ca/fhd.mp4
description:🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #roblox #роблокс
video_caption:A man and a woman are standing in front of a wall and talking to each other.
--------------------
link:https://cdn-st.rutubelist.ru/media/e9/e0/b47a9df14a5e97942715e5e705c0/fhd.mp4
description:#boobs , #красивыедевушки , #ass
video_caption:A young woman and a young man are posing for a picture.
--------------------
link:https://cdn-st.rutubelist.ru/media/87/43/b11df3f344d0af773aac81e410ee/fhd.mp4
description:nan
video_caption:A woman is sitting at a table holding a card and talking about it.
--------------------
link:https://cdn-st.rutubelist.ru/me

### [movie-picture-captioning](https://huggingface.co/dumperize/movie-picture-captioning)

In [ ]:
from PIL import Image
import torch
from transformers import AutoTokenizer, ViTFeatureExtractor, VisionEncoderDecoderModel
import av

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dumperize/movie-picture-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("dumperize/movie-picture-captioning")
model = VisionEncoderDecoderModel.from_pretrained("dumperize/movie-picture-captioning")

tokenizer_config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

In [ ]:
max_length = 128
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
video_captions = []
for video_path in tqdm_notebook(sorted_video_files[:5]):
    # load video
    container = av.open(video_path)

    # get fps (corrected approach)
    fps = container.streams.video[0].average_rate.numerator / container.streams.video[0].average_rate.denominator

    # extract frames at 25%, 50%, & 75%
    seg_len = container.streams.video[0].frames  # total number of frames
    indices = [int(0.25 * seg_len), int(0.5 * seg_len), int(0.75 * seg_len)]
    frames = []

    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i in indices:
          frames.append(frame.to_ndarray(format="rgb24"))
    # generate caption
    gen_kwargs = {
        "min_length": 0,
        "max_length": 100,
        "num_beams": 4,
    }

    pixel_values = feature_extractor(images=frames, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    caption = '|'.join([pred.strip() for pred in preds])
    video_captions.append(caption)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
for index in range(len(sorted_video_files[:5])):
    row = df.iloc[index]
    print(f'link:{row.link}')
    print(f'description:{row.description}')
    print(f'video_caption:{video_captions[index]}')
    print(20*'-')

link:https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4
description:#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin
video_caption:В основе сюжета фильма лежит история тренерской команды средней школы в парнях которого пятнадцатилетней давности, а тренер и герой фильма ( в прошлом - первая женщина - тренер ) Кэти, в настоящее время одинокий и помешанный на всю жизнь тренер. Кэти мечтает о спортивной карьере, но в итоге фильма попадает в руки бывшего тренера, а он, в итоге, меняет полностью свою жизнь.|В основе сюжета фильма лежит история игроков, помешанных на становлении легендарного вида спорта спорта. После несчастного случая во время игры в названием « Скрэбдинг », в 1992 году команда решает принять участие в чемпионате мира по футболу.|Фильм - портрет о том, как мы воображаем и изображаем смерть. Герои фильма живут в разных странах мира и ( в том числе и в России ). У каждого из нас есть своя история и свои правила

### проба моделей, которые достают текст из скринов

### проба моделей audio2text [whishper hf link](https://huggingface.co/openai)

In [ ]:
# !pip install

In [ ]:
# !pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip
import whisper

In [ ]:
# Шаг 1: Извлечение аудио из видео и сохранение его в формате MP3
# Создаем директорию для сохранения аудио файлов
audio_output_dir = "audio_files"
os.makedirs(audio_output_dir, exist_ok=True)

for video_file in sorted_video_files[:10]:
    video_clip = VideoFileClip(video_file)
    audio_file_path = os.path.join(audio_output_dir, os.path.basename(video_file).replace(".mp4", ".mp3"))
    video_clip.audio.write_audiofile(audio_file_path)

chunk:  86%|████████████████████████████████████████████████████████████████████████████▉            | 351/406 [00:29<00:00, 1309.79it/s, now=None]

MoviePy - Writing audio in audio_files/0_b0_e9_ef285e0241139fc611318ed33071_fhd.mp3



chunk:  86%|████████████████████████████████████████████████████████████████████████████▉            | 351/406 [00:29<00:00, 1309.79it/s, now=None]

MoviePy - Done.


chunk:  86%|████████████████████████████████████████████████████████████████████████████▉            | 351/406 [00:29<00:00, 1309.79it/s, now=None]

...

MoviePy - Done.


chunk:  86%|████████████████████████████████████████████████████████████████████████████▉            | 351/406 [00:34<00:00, 1309.79it/s, now=None]

MoviePy - Writing audio in audio_files/9_fc_73_d86600d04e519178a55774dfe65e_fhd.mp3



chunk:  86%|████████████████████████████████████████████████████████████████████████████▉            | 351/406 [00:35<00:00, 1309.79it/s, now=None]

MoviePy - Done.


In [ ]:
# Загружаем модель Whisper
model = whisper.load_model("small")

# Транскрибация аудио файлов
audio_files = [os.path.join(audio_output_dir, f) for f in os.listdir(audio_output_dir) if f.endswith(".mp3")]

100%|███████████████████████████████████████| 461M/461M [01:07<00:00, 7.20MiB/s]


In [ ]:
sorted_audio_files = sorted(audio_files, key=lambda x: int(os.path.basename(x).split('_')[0]))

In [ ]:
transcriptions = []
results_audio = []
for audio_file in tqdm_notebook(sorted_audio_files[:10]):
    # Транскрибация аудио
    result = model.transcribe(audio_file, language='ru')
    results_audio.append(result)
    transcriptions.append(result["text"])

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
for index in range(len(sorted_audio_files[:10])):
    row = df.iloc[index]
    print(index)
    print(f'link:{row.link}')
    print(f'description:{row.description}')
    print(f'transcription:{transcriptions[index]}')
    print(20*'-')

0
link:https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4
description:#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin
transcription: Субтитры мы SimilarlyStory
--------------------
1
link:https://cdn-st.rutubelist.ru/media/39/6c/b31bc6864bef9d8a96814f1822ca/fhd.mp4
description:🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #roblox #роблокс
transcription: Сейчас я тебе покажу секретную команду в Roblox'е! Чтобы её активировать поставь лайк и подпишись, а также в виде в чат команду I love you! Когда вы её ведёте у вас на экране появится вот такой клоун! Скривц был создан одним из создателей Roblox'а и работает только в эракде есть Roblox Developer Service. Чтобы в Раскриме ранжа всего-то выйти из игры, но лучше не проверяй!
--------------------
2
link:https://cdn-st.rutubelist.ru/media/e9/e0/b47a9df14a5e97942715e5e705c0/fhd.mp4
description:#boobs , #красивыедевушки , #ass
transcription: Смотрите такие игрики в один раз.
-----

# Идеи полей

- синонимы для хэштего: можно взять совстречаемость тегов и попробовать распарсить какой-нибудь словарь синонимов
- Классификация языка аудио, чтобы потом asr нужного языка применять
- (+) транскрибация аудио в текст (audio to text) - вишпер
- (+) хэштеги по тексту - text classification, text summarization, chatgpt
- (+) шазам, если есть известная песня, выводим в хэштег ее название - shazamio
- транскрибация изображения, если есть текст (image to text)
- хэштеги по фото (по первому, среднему и последнему) - image classification / image captioning
- распознавание известных личностей по фото (милохин, мелстрой и тд)

In [ ]:
!pip install shazamio

  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
Using cached anyio-4.3.0-py3-none-any.whl (85 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.6/904.6 kB 1.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: anyio
    Found existing installation: anyio 4.2.0
    Uninstalling anyio-4.2.0:
      Successfully uninstalled anyio-4.2.0


In [ ]:
import asyncio
from shazamio import Shazam
import nest_asyncio
# Разрешаем повторный запуск цикла событий в Jupyter Notebook
nest_asyncio.apply()

async def recognize_audio(file_path):
    shazam = Shazam()
    out = await shazam.recognize_song(file_path)
    return out

In [ ]:
file_path = sorted_audio_files[7]
file_path

'audio_files/7_19_9c_98de3b4a4d9e83cb5dda828f899e_fhd.mp3'

In [ ]:
loop = asyncio.get_event_loop()
result = loop.run_until_complete(recognize_audio(file_path))

# Вывод информации о песне
if result and 'track' in result:
    track = result['track']
    print(f"Title: {track['title']}")
    print(f"Subtitle: {track['subtitle']}")
    print(f"URL: {track['url']}")
else:
    print("Song not recognized")

Title: What Was I Made For? (From The Motion Picture "Barbie")
Subtitle: Billie Eilish
URL: https://www.shazam.com/track/667426645/what-was-i-made-for-from-the-motion-picture-barbie
